# Data Cleaning

Note: There is an old script called `data-cleaning-old.ipynb`. This is old and not used anymore.

In [1]:
import numpy as np
import pandas as pd
import pickle
import tempfile

## Game Data

In [2]:
# This function is to convert the string into int
def Str2float(string):
    try:
        return float(string)
    except:
        return "NaN"

In [3]:
#Construct the game stats with each team rosters
#import 2016
game16_df = pd.read_csv('raw-data/15-16game_stats.csv')[["Visitor/Neutral", "PTS", "Home/Neutral", "PTS.1"]]
game16_df['Year'] = 2016
game16_df['PTS'] = game16_df['PTS'].map(lambda x: Str2float(x))
game16_df['PTS.1'] = game16_df['PTS.1'].map(lambda x: Str2float(x))

# get rid of nan or nonsensical values
game16_df = game16_df[game16_df['PTS'].map(lambda x: type(x) is float)]

# difference between visitor and home
game16_df['diff'] = game16_df['PTS'] - game16_df['PTS.1']

In [4]:
# make a Visitor/Neutral and Home/Neutral index
teams_set = set(game16_df['Visitor/Neutral'])

team_dict = {team:i for (i, team) in enumerate(teams_set)}

game16_df['Visitor_Index'] = game16_df['Visitor/Neutral'].apply(lambda x: team_dict[x])
game16_df['Home_Index'] = game16_df['Home/Neutral'].apply(lambda x: team_dict[x])

In [5]:
game16_df.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Year,diff,Visitor_Index,Home_Index
1,Detroit Pistons,106,Atlanta Hawks,94,2016,12,5,26
2,Cleveland Cavaliers,95,Chicago Bulls,97,2016,-2,25,11
3,New Orleans Pelicans,95,Golden State Warriors,111,2016,-16,24,10
4,Philadelphia 76ers,95,Boston Celtics,112,2016,-17,4,6
5,Chicago Bulls,115,Brooklyn Nets,100,2016,15,11,27


In [6]:
game16_df.to_csv("clean-data/game_outcomes_15-16.csv", index = False)

## Player Data

In [7]:
player_df = pd.read_csv("raw-data/15-16Per100Pos_stats.csv")
player_df.drop('Unnamed: 29', axis=1, inplace=True)
player_df.drop('Rk', axis=1, inplace=True)

In [8]:
player_df = player_df[player_df['Player'] != 'Player']
player_df = player_df.drop_duplicates(subset = "Player")

In [9]:
player_df.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg
0,Quincy Acy,PF,25,SAC,59,29,876,6.5,11.7,.556,...,6.7,10.3,1.5,1.6,1.3,1.5,5.6,16.8,124,108
1,Jordan Adams,SG,21,MEM,2,0,15,6.9,20.6,.333,...,6.9,6.9,10.3,10.3,0.0,6.9,6.9,24.0,84,90
2,Steven Adams,C,22,OKC,80,80,2014,6.4,10.5,.613,...,7.7,13.1,1.5,1.0,2.2,2.1,5.5,15.7,123,105
3,Arron Afflalo,SG,30,NYK,71,57,2371,7.7,17.3,.443,...,5.3,5.8,3.1,0.5,0.2,1.8,3.1,19.7,105,112
4,Alexis Ajinca,C,27,NOP,59,17,861,8.6,18.1,.476,...,11.2,15.5,1.8,1.1,2.1,3.1,7.7,20.3,100,107


In [10]:
#import name and their teams 
playerTeam_df = pd.read_csv('raw-data/salary.csv', header = 1)[['Player','Tm','2015-16']]

# get rid of nonsensical values
playerTeam_df = playerTeam_df[playerTeam_df['Tm'] != 'Tm']
playerTeam_df = playerTeam_df[playerTeam_df['Player'] != 'Player']
playerTeam_df = playerTeam_df[playerTeam_df['2015-16'] != 'Salary']
# convert to integer
playerTeam_df['2015-16'] = playerTeam_df['2015-16'].map(lambda x: int(x))
# only select one salary per player
salary_df = pd.DataFrame(playerTeam_df.groupby('Player')['2015-16'].max())
salary_df["Player"] = salary_df.index
salary_df = salary_df.reset_index(drop=True)

playerTeam_df = playerTeam_df[~pd.isnull(playerTeam_df['Player'])]
playerTeam_df =  playerTeam_df.drop_duplicates('Player')

# merge
playerTeam_df = playerTeam_df[["Player","Tm"]].merge(salary_df,how="inner", on="Player")
playerTeam_df.head()

,Player,Tm,2015-16
0,Kobe Bryant,Los Angeles Lakers,25000000
1,Joe Johnson,Brooklyn Nets,24894863
2,LeBron James,Cleveland Cavaliers,22971000
3,Carmelo Anthony,New York Knicks,22875000
4,Dwight Howard,Houston Rockets,22359364


In [11]:
# merge all the player info
player_info_df = playerTeam_df.merge(player_df,how='inner',on='Player')
player_info_df.drop(u"Tm_y", axis = 1, inplace=True)
player_info_df.rename(columns={"Tm_x": "Tm"}, inplace=True)

In [12]:
# save file to temporary csv and read it back so we can properly convert stuff to floats
tempfp = tempfile.NamedTemporaryFile()
player_info_df.to_csv(tempfp, index = False)
player_info_df = pd.read_csv(tempfp.name)
tempfp.close()

In [13]:
player_info_df = player_info_df[~pd.isnull(player_info_df['ORtg'])]
player_info_df.head()

,Player,Tm,2015-16,Pos,Age,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg
0,Kobe Bryant,Los Angeles Lakers,25000000,SF,37,66,66,1863,10.7,30.0,...,5.6,6.7,5.0,1.7,0.4,3.5,3.1,31.3,95.0,113.0
1,Joe Johnson,Brooklyn Nets,24894863,SF,34,81,81,2703,7.1,16.1,...,4.6,5.5,6.0,1.1,0.1,3.0,2.7,18.6,104.0,113.0
2,LeBron James,Cleveland Cavaliers,22971000,SF,31,76,76,2709,14.0,26.9,...,8.6,10.7,9.8,2.0,0.9,4.7,2.7,36.5,116.0,103.0
3,Carmelo Anthony,New York Knicks,22875000,SF,31,72,72,2530,11.5,26.6,...,9.3,11.3,6.1,1.3,0.8,3.6,3.7,32.0,107.0,107.0
4,Dwight Howard,Houston Rockets,22359364,C,30,71,71,2280,8.0,12.9,...,12.9,18.0,2.1,1.5,2.4,3.6,4.7,21.0,111.0,104.0


We see now that the values are properly converted to floats/ints.

In [14]:
player_info_df['ORtg'].values[:10]

array([  95.,  104.,  116.,  107.,  111.,  114.,  119.,  122.,   96.,  103.])

In [15]:
# Save the info to CSV format
player_info_df.to_csv("clean-data/player_info.csv", index = False)

The `2015-2016` column refers to the salary.

## Masking

The following part is guest/host_team_line_up. `guest_team_line_up` and `host_team_line_up` are lists of boolean Numpy arrays. Each element of each list corresponds to one matchup in the `game16_df` table. `guest_team_line_up` can be used to select the players from the `player_info_df` table that participated on the guest team, and `host_team_line_up` can be used to select the players from `player_info_df` that participated for the host.

In [16]:
players_teams = {}
gb = player_info_df.groupby("Tm")
for x in gb.groups:
    players_teams[x] = gb.get_group(x)

In [17]:
#Convert the player into the index look-up list
player_idx = player_info_df['Player'].values.tolist() 
roster_idx = {}
for team in players_teams:
    roster_idx[team] = [player_idx.index(i) for i in players_teams[team]['Player'].values]

In [18]:
guest_teams = game16_df['Visitor/Neutral'].values 
host_teams = game16_df['Home/Neutral'].values 

In [19]:
player_num = len(player_idx)

In [20]:
#Convert guest team to an array of player's indicator
guest_team_line_up = []
for team in guest_teams:
    # Initialize the binary number
    player_indicator = np.array([False] * player_num)
    # Here I have the index of player
    list_idx = roster_idx[team]
    for i in list_idx:
        player_indicator[i] = True
    guest_team_line_up.append(player_indicator)
print "we have total:",len(guest_team_line_up)
print "confirm the previous num", len(guest_teams)

we have total: 1230
confirm the previous num 1230


In [21]:
#Convert guest team to an array of player's indicator
host_team_line_up = []
for team in host_teams:
    # Initialize the binary number
    player_indicator = np.array([False] * player_num)
    # Here I have the index of player
    list_idx = roster_idx[team]
    for i in list_idx:
        player_indicator[i] = True 
    host_team_line_up.append(player_indicator)
#     print len(beta)
#     print len(host_team_line_up[0])
#     check = beta[host_team_line_up[0]]
#     break
print "we have total:",len(host_team_line_up)
print "confirm the previous num", len(host_teams)

we have total: 1230
confirm the previous num 1230


In [22]:
with open("clean-data/guest_team_line_up.pkl", "wb") as f:
    pickle.dump(guest_team_line_up, f)
    
with open("clean-data/host_team_line_up.pkl", "wb") as f:
    pickle.dump(host_team_line_up, f)

Example of how to use these:

In [23]:
player_info_df[guest_team_line_up[0]]

,Player,Tm,2015-16,Pos,Age,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg
23,Tobias Harris,Detroit Pistons,16000000,PF,23,76,74,2513,8.3,17.7,...,8.3,10.2,3.4,1.3,0.8,2.2,3.1,22.3,112.0,107.0
39,Reggie Jackson,Detroit Pistons,14000000,PG,25,79,79,2424,11.2,25.9,...,4.1,5.3,10.2,1.2,0.2,4.6,3.9,31.0,109.0,108.0
117,Aron Baynes,Detroit Pistons,6500000,C,29,81,1,1233,7.9,15.7,...,10.0,15.7,2.1,0.9,2.1,2.7,6.2,21.0,114.0,104.0
122,Jodie Meeks,Detroit Pistons,6270000,SG,28,3,0,43,8.2,23.5,...,4.7,5.9,3.5,0.0,0.0,1.2,2.3,25.8,111.0,111.0
141,Josh Smith,Detroit Pistons,5400000,PF,30,55,7,880,7.2,19.7,...,8.0,10.8,5.1,1.9,2.7,4.3,6.7,18.7,88.0,103.0
156,Marcus Morris,Detroit Pistons,5000000,SF,26,80,80,2856,7.2,16.7,...,5.5,7.1,3.6,1.2,0.4,2.5,3.0,20.0,107.0,108.0
209,Andre Drummond,Detroit Pistons,3272091,C,22,81,81,2665,10.5,20.1,...,15.2,22.7,1.3,2.3,2.1,2.9,4.6,24.9,103.0,98.0
225,Anthony Tolliver,Detroit Pistons,3000000,PF,30,72,5,1341,4.8,12.5,...,7.2,8.6,2.0,1.0,0.6,1.1,3.2,14.4,113.0,107.0
229,Kentavious Caldwell-Pope,Detroit Pistons,2891760,SG,22,76,76,2789,7.3,17.4,...,3.8,5.1,2.5,2.0,0.3,1.9,3.0,20.0,107.0,107.0
235,Stanley Johnson,Detroit Pistons,2841960,SF,19,73,6,1687,6.7,18.0,...,7.4,9.2,3.5,1.6,0.4,3.4,5.3,17.7,91.0,106.0


In [24]:
player_info_df[host_team_line_up[0]]

,Player,Tm,2015-16,Pos,Age,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg
15,Paul Millsap,Atlanta Hawks,19000000,PF,30,81,81,2647,9.4,19.9,...,10.0,13.7,4.9,2.7,2.6,3.6,4.4,25.9,108.0,96.0
61,Al Horford,Atlanta Hawks,12000000,C,29,82,82,2631,9.9,19.7,...,8.4,11.2,4.9,1.3,2.3,2.0,3.1,23.5,113.0,101.0
87,Tiago Splitter,Atlanta Hawks,8500000,C,31,36,2,579,6.9,13.2,...,6.4,10.2,2.6,1.7,1.0,2.0,6.1,17.2,113.0,102.0
96,Jeff Teague,Atlanta Hawks,8000000,PG,27,79,78,2255,9.5,21.7,...,4.0,4.7,10.3,2.1,0.5,4.8,3.7,27.2,105.0,104.0
127,Kyle Korver,Atlanta Hawks,5746479,SG,34,80,80,2401,5.5,12.7,...,5.1,5.4,3.4,1.2,0.7,2.0,3.3,15.2,104.0,105.0
184,Thabo Sefolosha,Atlanta Hawks,4000000,SF,31,75,11,1758,5.3,10.5,...,7.9,9.4,3.0,2.4,1.0,1.9,3.1,13.5,108.0,100.0
207,Mike Scott,Atlanta Hawks,3333333,PF,27,75,0,1146,7.6,16.2,...,6.6,8.9,3.4,1.1,0.7,1.8,4.4,20.0,112.0,104.0
230,Kirk Hinrich,Atlanta Hawks,2870000,PG,35,46,7,631,3.9,10.2,...,4.9,5.6,5.8,1.2,0.2,2.4,5.1,10.9,101.0,109.0
280,Kent Bazemore,Atlanta Hawks,2000000,SF,26,75,68,2083,7.6,17.2,...,8.3,9.0,4.1,2.3,0.9,3.3,4.2,20.7,99.0,100.0
287,Dennis Schroder,Atlanta Hawks,1763400,PG,22,80,6,1621,10.0,23.8,...,5.5,6.2,10.6,2.1,0.3,5.6,4.2,26.8,97.0,103.0
